In [1]:
from datetime import datetime
import pyspark.sql.functions as functions
from pyspark.sql.functions import col
import pandas as pd

In [2]:
import getpass
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("yarn") \
    .appName('twitter-{0}'.format(getpass.getuser())) \
    .config('spark.executor.memory', '4g') \
    .config('spark.executor.instances', '5') \
    .config('spark.port.maxRetries', '100') \
    .getOrCreate()

In [20]:
# load the data
df = spark.read.csv('/datasets/sbb/2018/03/2018-03-01istdaten.csv.bz2', sep=';', header=True)

In [21]:
stations = pd.read_csv('data/filtered_stations.csv')
valid_stations = set(stations['Remark'])

In [22]:
def is_valid(text):
    """Convert string to datetime"""
    return text in valid_stations

valid_stops = df.filter((df.DURCHFAHRT_TF=='false') & 
                        (df.FAELLT_AUS_TF=='false') & 
                        (df.ZUSATZFAHRT_TF=='false') &
                        (df.HALTESTELLEN_NAME.isin(valid_stations)))\
                .select('FAHRT_BEZEICHNER', 
                        'PRODUKT_ID', 
                        'LINIEN_TEXT', 
                        'HALTESTELLEN_NAME', 
                        'ANKUNFTSZEIT', 
                        'ABFAHRTSZEIT')\
                .withColumn('ANKUNFTSZEIT', functions.to_timestamp(df.ANKUNFTSZEIT, 'dd.MM.yyyy HH:mm'))\
                .withColumn('ABFAHRTSZEIT', functions.to_timestamp(df.ABFAHRTSZEIT, 'dd.MM.yyyy HH:mm'))

In [23]:
valid_stops.show(n=5)

+----------------+----------+-----------+-----------------+-------------------+-------------------+
|FAHRT_BEZEICHNER|PRODUKT_ID|LINIEN_TEXT|HALTESTELLEN_NAME|       ANKUNFTSZEIT|       ABFAHRTSZEIT|
+----------------+----------+-----------+-----------------+-------------------+-------------------+
|    85:11:10:002|       Zug|         EC|        Zürich HB|2018-03-01 21:51:00|               null|
|    85:11:11:001|       Zug|         EC|        Zürich HB|               null|2018-03-01 06:09:00|
|    85:11:12:001|       Zug|         EC|        Zürich HB|2018-03-01 10:51:00|               null|
|  85:11:1251:001|       Zug|        IC3|        Zürich HB|2018-03-01 07:00:00|               null|
|  85:11:1252:001|       Zug|         IC|        Zürich HB|2018-03-01 21:23:00|2018-03-01 21:36:00|
+----------------+----------+-----------+-----------------+-------------------+-------------------+
only showing top 5 rows



In [24]:
departures = valid_stops.filter(valid_stops.ABFAHRTSZEIT.isNotNull())\
                        .drop('ANKUNFTSZEIT')
arrivals   = valid_stops.filter(valid_stops.ANKUNFTSZEIT.isNotNull())\
                        .drop('ABFAHRTSZEIT')

In [25]:
departures.show(n=5)
arrivals.show(n=5)

+----------------+----------+-----------+-----------------+-------------------+
|FAHRT_BEZEICHNER|PRODUKT_ID|LINIEN_TEXT|HALTESTELLEN_NAME|       ABFAHRTSZEIT|
+----------------+----------+-----------+-----------------+-------------------+
|    85:11:11:001|       Zug|         EC|        Zürich HB|2018-03-01 06:09:00|
|  85:11:1252:001|       Zug|         IC|        Zürich HB|2018-03-01 21:36:00|
|  85:11:1255:001|       Zug|        IC3|        Zürich HB|2018-03-01 08:37:00|
|  85:11:1260:001|       Zug|        IC3|        Zürich HB|2018-03-01 21:00:00|
|    85:11:13:001|       Zug|         EC|        Zürich HB|2018-03-01 07:09:00|
+----------------+----------+-----------+-----------------+-------------------+
only showing top 5 rows

+----------------+----------+-----------+-----------------+-------------------+
|FAHRT_BEZEICHNER|PRODUKT_ID|LINIEN_TEXT|HALTESTELLEN_NAME|       ANKUNFTSZEIT|
+----------------+----------+-----------+-----------------+-------------------+
|    85:11:10:0

In [26]:
arrivals.createOrReplaceTempView('arrivals')
departures.createOrReplaceTempView('departures')

In [27]:
joinQuery = 'SELECT d.HALTESTELLEN_NAME AS src, a.HALTESTELLEN_NAME AS dst,              \
                    d.PRODUKT_ID AS type, d.LINIEN_TEXT AS subtype, \
                    d.ABFAHRTSZEIT AS departure_time, a.ANKUNFTSZEIT AS arrival_time     \
             FROM arrivals AS a INNER JOIN departures AS d                               \
             ON a.FAHRT_BEZEICHNER == d.FAHRT_BEZEICHNER                                 \
             WHERE a.HALTESTELLEN_NAME != d.HALTESTELLEN_NAME                            \
             AND d.ABFAHRTSZEIT < a.ANKUNFTSZEIT'

edges = spark.sql(joinQuery)

In [28]:
edges.show()

+----------------+----------------+----+-------+-------------------+-------------------+
|             src|             dst|type|subtype|     departure_time|       arrival_time|
+----------------+----------------+----+-------+-------------------+-------------------+
|       Zürich HB|Zürich Flughafen| Zug|    IC5|2018-03-01 06:39:00|2018-03-01 06:49:00|
|       Zürich HB|Zürich Flughafen| Zug|    IC5|2018-03-01 07:39:00|2018-03-01 07:49:00|
|Zürich Flughafen|       Zürich HB| Zug|    IC5|2018-03-01 07:13:00|2018-03-01 07:23:00|
|       Zürich HB|Zürich Flughafen| Zug|    IC5|2018-03-01 08:39:00|2018-03-01 08:49:00|
|Zürich Flughafen|       Zürich HB| Zug|    IC5|2018-03-01 08:13:00|2018-03-01 08:23:00|
|       Zürich HB|Zürich Flughafen| Zug|    IC5|2018-03-01 09:39:00|2018-03-01 09:49:00|
|Zürich Flughafen|       Zürich HB| Zug|    IC5|2018-03-01 09:13:00|2018-03-01 09:23:00|
|       Zürich HB|Zürich Flughafen| Zug|    IC5|2018-03-01 10:39:00|2018-03-01 10:49:00|
|Zürich Flughafen|   

In [29]:
edges.count()

2153164

In [61]:
grouped = edges.groupBy([edges.src, edges.dst, edges.type, edges.subtype])
grouped_edges = grouped.agg({'departure_time': 'collect_list',
                             'arrival_time'  : 'collect_list'})\
                        .withColumnRenamed('collect_list(arrival_time)', 'arrival_times')\
                        .withColumnRenamed('collect_list(departure_time)', 'departure_times')

In [56]:
grouped_edges.count()

32979

In [62]:
grouped_edges.show()

+--------------------+--------------------+----+-------+--------------------+--------------------+
|                 src|                 dst|type|subtype|       arrival_times|     departure_times|
+--------------------+--------------------+----+-------+--------------------+--------------------+
|Bassersdorf, Spor...|Dietlikon, Bahnho...| Bus|    765|[2018-03-01 16:54...|[2018-03-01 16:39...|
|Bassersdorf, Talg...|  Kloten, Kirchgasse| Bus|    765|[2018-03-01 17:23...|[2018-03-01 17:17...|
|Birmensdorf ZH, B...|Zürich, Goldbrunn...| Bus|    215|[2018-03-01 15:38...|[2018-03-01 15:20...|
|Bonstetten, Dorfp...|Wettswil a.A., Mo...| Bus|    210|[2018-03-01 07:16...|[2018-03-01 07:08...|
| Bonstetten-Wettswil|           Zürich HB| Zug|    S14|[2018-03-01 06:40...|[2018-03-01 06:18...|
|Bonstetten-Wettsw...|Wettswil a.A., Bä...| Bus|    221|[2018-03-01 17:48...|[2018-03-01 17:33...|
|Dietlikon, Hofwiesen|Dietlikon, Dornen...| Bus|    749|[2018-03-01 06:09...|[2018-03-01 06:06...|
|Dietlikon

In [63]:
grouped_edges.write.parquet('/homes/schmutz/edges', mode='overwrite')

In [65]:
#test_df = spark.read.parquet('/homes/schmutz/edges')